# Preprocessing Exercises

In [1]:
#data manipulation
import pandas as pd
import numpy as np

#stats is great
from scipy import stats

from sklearn.model_selection import train_test_split

#my own files with my own functions
import acquire as a
import prepare as p

import os
import env

Do these exercises in a notebook called modeling.ipynb first, then transfer the final functions to the model.py file.

This work should all be saved in your local classification-exercises repo. Add, commit, and push your changes.

# Using the Titanic dataset

In [2]:
p.splitting_data?
###run it to see details

Signature: p.splitting_data(df, col, seed=123)
Docstring:
Just like the splitting Titanic function but it can be used for any df now!
must provide the df and column. Does not clean it though
File:      ~/Codeup-Data-Science/classification-exercises/prepare.py
Type:      function

### 1. Use the function defined in acquire.py to load the Titanic data.

### 2. Use the function defined in prepare.py to prepare the titanic data.

In [3]:
# p.splitting_data(
#     p.clean_titanic(a.get_titanic_data()),
#     'survived')

this file exists, reading csv


In [3]:
# grab, prepare, and split our data sets
titanic_train, \
titanic_val, \
titanic_test = p.splitting_data(
    p.clean_titanic(a.get_titanic_data()),
    'survived')

this file exists, reading csv


In [4]:
# do the same thing for telco
telco_train, \
telco_val, \
telco_test = p.splitting_data(
    p.prep_telco(a.get_telco_data()),
    'churn')

this file exists, reading csv


In [5]:
telco_train.shape, telco_val.shape, telco_test.shape

((4225, 21), (1409, 21), (1409, 21))

In [6]:
### remember I added back the age column and removed rows with null values instead
titanic_train.shape, titanic_val.shape, titanic_test.shape

((427, 10), (142, 10), (143, 10))

## 3. Encode the categorical columns on train dataset. Create dummy variables of the categorical columns and concatenate them onto the dataframe. Remove the columns they are replacing. Repeat on validate and test.

In [8]:
# titanic preprocessing:
# we saw previously that the things that we need to encode are:
# sex (nominal, binary)
# and embark_town (nominal, multicategorical/nonbinary)

In [7]:
titanic_train.pclass.astype(int)

702    3
199    2
108    3
872    1
827    2
      ..
640    3
476    2
217    2
67     3
452    1
Name: pclass, Length: 427, dtype: int64

In [8]:
titanic_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 427 entries, 702 to 452
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   passenger_id  427 non-null    int64  
 1   survived      427 non-null    int64  
 2   pclass        427 non-null    object 
 3   sex           427 non-null    object 
 4   age           427 non-null    float64
 5   sibsp         427 non-null    int64  
 6   parch         427 non-null    int64  
 7   fare          427 non-null    float64
 8   embark_town   427 non-null    object 
 9   alone         427 non-null    int64  
dtypes: float64(2), int64(5), object(3)
memory usage: 36.7+ KB


In [9]:
# tidy up the pclass and drop passenger_id:
titanic_train.drop(columns='passenger_id')
titanic_train['pclass'] = titanic_train['pclass'].astype(int)

In [10]:
# encoding sex and embark_town,
# the only object types we have left:

In [11]:
# let's assign the output of pd.getdummies into a separate df:
titanic_train_encoded_cats = pd.get_dummies(titanic_train[['embark_town', 'sex']],
              drop_first=True).astype(int)

In [12]:
# glue that back into my original df:
titanic_preprocessed = pd.concat(
    [titanic_train, 
    titanic_train_encoded_cats],
    axis=1).drop(columns=['sex', 'embark_town'])

In [13]:
titanic_preprocessed.head()

,passenger_id,survived,pclass,age,sibsp,parch,fare,alone,embark_town_Queenstown,embark_town_Southampton,sex_male
702,702,0,3,18.0,0,1,14.4542,0,0,0,0
199,199,0,2,24.0,0,0,13.0000,1,0,1,0
108,108,0,3,38.0,0,0,7.8958,1,0,1,1
872,872,0,1,33.0,0,0,5.0000,1,0,1,1
827,827,1,2,1.0,0,2,37.0042,0,0,0,1


## 4. Create a function named preprocess_titanic that accepts the train, validate, and test titanic data, and returns the dataframes ready for modeling.

In [14]:
def preprocess_titanic(train_df, val_df, test_df):
    '''
    preprocess_titanic will take in three pandas dataframes
    of our titanic data, expected as cleaned versions of this 
    titanic data set (see documentation on acquire.py and prepare.py)
    
    output:
    encoded, ML-ready versions of our clean data, with 
    columns sex and embark_town encoded in the one-hot fashion
    return: (pd.DataFrame, pd.DataFrame, pd.DataFrame)
    '''
    
    # with a looping structure:
    # for df in [train_df, val_df, test_df]:
    #     df.drop(blah blah blah)
    #     df['pclass'] = df['pclass'].astype(int)
    train_df = train_df.drop(columns='passenger_id')
    train_df['pclass'] = train_df['pclass'].astype(int)
    val_df = val_df.drop(columns='passenger_id')
    val_df['pclass'] = val_df['pclass'].astype(int)
    test_df = test_df.drop(columns='passenger_id')
    test_df['pclass'] = test_df['pclass'].astype(int)
    encoding_var = ['sex', 'embark_town']
    encoded_dfs = []
    for df in [train_df, val_df, test_df]:
        df_encoded_cats = pd.get_dummies(
            df[['embark_town', 'sex']],
              drop_first=True).astype(int)
        encoded_dfs.append(pd.concat(
            [df,
            df_encoded_cats],
            axis=1).drop(columns=['sex', 'embark_town']))
    return encoded_dfs

In [15]:
ttrain_encoded, tval_encoded, ttest_encoded = preprocess_titanic(
    titanic_train, 
    titanic_val, 
    titanic_test)

In [16]:
# grabbing info:
# for exploration:
# splitting_data(clean_titanic(acquire_titanic()))

# for modeling:
# preprocess_titanic(splitting_data(clean_titanic(acquire_titanic))

# Using the Telco dataset

In [35]:
telco_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4225 entries, 2865 to 5354
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   customer_id            4225 non-null   object 
 1   gender                 4225 non-null   object 
 2   senior_citizen         4225 non-null   int64  
 3   partner                4225 non-null   object 
 4   dependents             4225 non-null   object 
 5   tenure                 4225 non-null   int64  
 6   phone_service          4225 non-null   object 
 7   multiple_lines         4225 non-null   object 
 8   online_security        4225 non-null   object 
 9   online_backup          4225 non-null   object 
 10  device_protection      4225 non-null   object 
 11  tech_support           4225 non-null   object 
 12  streaming_tv           4225 non-null   object 
 13  streaming_movies       4225 non-null   object 
 14  paperless_billing      4225 non-null   object 
 15  monthl

In [38]:
telco_train.senior_citizen

2865    1
4107    1
3453    1
923     0
2370    0
       ..
1286    0
4999    1
6892    0
3718    0
5354    0
Name: senior_citizen, Length: 4225, dtype: int64

### 1. Use the function defined in acquire.py to load the Telco data.

### 2. Use the function defined in prepare.py to prepare the Telco data.

In [20]:
# # do the same thing for telco ### DID THIS ABOVE^
# telco_train, \
# telco_val, \
# telco_test = p.splitting_data(
#     p.prep_telco(a.get_telco_data()),
#     'churn')

### 3. Encode the categorical columns on train.

In [18]:
telco_train.select_dtypes('O') ### I wonder why total_charges is on Object

,customer_id,gender,partner,dependents,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,total_charges,churn,contract_type,internet_service_type,payment_type
2865,4083-BFNYK,Female,Yes,No,Yes,Yes,Yes,Yes,No,No,No,Yes,No,3591.25,No,One year,Fiber optic,Credit card (automatic)
4107,5804-LEPIM,Female,No,No,Yes,No,No,No,No,No,No,No,Yes,139.05,Yes,Month-to-month,Fiber optic,Electronic check
3453,4895-TMWIR,Male,Yes,No,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No,214.75,Yes,Month-to-month,NaN,Mailed check
923,1342-JPNKI,Male,No,No,Yes,Yes,No,No,No,No,Yes,No,Yes,834.1,Yes,Month-to-month,Fiber optic,Bank transfer (automatic)
2370,3397-AVTKU,Male,No,No,Yes,No,No,Yes,No,No,No,Yes,Yes,2548.55,No,Two year,DSL,Electronic check
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1286,1915-IOFGU,Female,No,No,Yes,No,No,No,No,No,No,No,No,70.5,Yes,Month-to-month,Fiber optic,Electronic check
4999,7025-WCBNE,Male,No,No,Yes,Yes,No,Yes,No,Yes,No,No,No,2754,No,Two year,DSL,Bank transfer (automatic)
6892,9788-HNGUT,Male,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,8594.4,No,Two year,Fiber optic,Credit card (automatic)
3718,5229-PRWKT,Male,No,No,Yes,Yes,No,No,No,No,No,Yes,Yes,649.65,Yes,Month-to-month,Fiber optic,Electronic check


In [19]:
# checking fields for object type:
# initialize some empty lists:
cat_cols, num_cols = [], []
for col in telco_train.columns:
    if telco_train[col].dtype == 'O':
        cat_cols.append(col)
    else:
        num_cols.append(col)

In [20]:
cat_cols

['customer_id',
 'gender',
 'partner',
 'dependents',
 'phone_service',
 'multiple_lines',
 'online_security',
 'online_backup',
 'device_protection',
 'tech_support',
 'streaming_tv',
 'streaming_movies',
 'paperless_billing',
 'total_charges',
 'churn',
 'contract_type',
 'internet_service_type',
 'payment_type']

In [21]:
cat_cols.remove('customer_id')

In [22]:
cat_cols

['gender',
 'partner',
 'dependents',
 'phone_service',
 'multiple_lines',
 'online_security',
 'online_backup',
 'device_protection',
 'tech_support',
 'streaming_tv',
 'streaming_movies',
 'paperless_billing',
 'total_charges',
 'churn',
 'contract_type',
 'internet_service_type',
 'payment_type']

#### a. Encode at least one column using .replace
* great for binary categories that are nominal: True:False switches

In [26]:
# pd.Series.replace?

In [23]:
telco_train.loc[:, 'is_female'] = telco_train['gender'].replace({'Female': 1, 'Male': 0})

In [24]:
telco_train.partner.unique()

array(['Yes', 'No'], dtype=object)

#### b. Encode at least one column using .map
* great for binary categories that are nominal: True:False switches

In [25]:
telco_train.loc[:, 'has_partner'] = telco_train['partner'].map({'Yes': 1, 'No': 0})

In [26]:
# telco_train = telco_train.set_index('customer_id')
# telco_train

In [27]:
# debugging for the total_charged that turns out was not already a number:

In [28]:
telco_train['total_charges'] = (telco_train.total_charges + '0').astype(float)
### asigns any missing values 0 before turning them into a float.
### existing digits in the form of strings won't be affected by that extra zero (mathematically)

In [29]:
telco_train.total_charges.min()

0.0

#### c. Encode the rest of the columns by creating dummy variables and concatenating them onto the dataframe.

### 4. Repeat the same steps on validate and test.

### 5. Create a function named prep_telco that accepts the train, validate, and test telco data, and returns the dataframes ready for modeling.

In [30]:
def preprocess_telco(train_df, val_df, test_df):
    '''
    preprocess_telco will take in three pandas dataframes
    of our telco data, expected as cleaned versions of this 
    telco data set (see documentation on acquire.py and prepare.py)
    
    output:
    encoded, ML-ready versions of our clean data, with 
    columns sex and embark_town encoded in the one-hot fashion
    return: (pd.DataFrame, pd.DataFrame, pd.DataFrame)
    '''
    # with a looping structure:
    # go through the three dfs, set the index to customer id
    for df in [train_df, val_df, test_df]:
        df = df.set_index('customer_id')
        df['total_charges'] = df['total_charges'].astype(float)
    # initialize an empty list to see what needs to be encoded:
    encoding_vars = []
    # loop through the columns to fill encoded_vars with appropriate
    # datatype field names
    for col in train_df.columns:
        if train_df[col].dtype == 'O':
            encoding_vars.append(col)
    encoding_vars.remove('customer_id')
    # initialize an empty list to hold our encoded dataframes:
    encoded_dfs = []
    for df in [train_df, val_df, test_df]:
        df_encoded_cats = pd.get_dummies(
            df[encoding_vars],
              drop_first=True).astype(int)
        encoded_dfs.append(pd.concat(
            [df,
            df_encoded_cats],
            axis=1).drop(columns=encoding_vars))
    return encoded_dfs

In [31]:
tlco_train_encoded, \
tlco_val_encoded, \
tlco_test_encoded = preprocess_telco(telco_train, telco_val, telco_test)

In [33]:
tlco_train_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4225 entries, 2865 to 5354
Data columns (total 33 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   customer_id                            4225 non-null   object 
 1   senior_citizen                         4225 non-null   int64  
 2   tenure                                 4225 non-null   int64  
 3   monthly_charges                        4225 non-null   float64
 4   total_charges                          4225 non-null   float64
 5   is_female                              4225 non-null   int64  
 6   has_partner                            4225 non-null   int64  
 7   gender_Male                            4225 non-null   int64  
 8   partner_Yes                            4225 non-null   int64  
 9   dependents_Yes                         4225 non-null   int64  
 10  phone_service_Yes                      4225 non-null   int64  
 11  multip

In [34]:
tlco_train_encoded.head(1).T

,2865
customer_id,4083-BFNYK
senior_citizen,1
tenure,38
monthly_charges,95.0
total_charges,3591.25
is_female,1
has_partner,1
gender_Male,0
partner_Yes,1
dependents_Yes,0
